#Final Assignment
ntroduction where you discuss the business problem and who would be interested in this project.

Data where you describe the data that will be used to solve the problem and the source of the data.

Methodology section which represents the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

Results section where you discuss the results.

Discussion section where you discuss any observations you noted and any recommendations you can make based on the results.

Conclusion section where you conclude the report.

In [25]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import requests
import time
%pip install geocoder
import geocoder

     |████████████████████████████████| 102kB 2.0MB/s 


In [ ]:
df = pd.read_csv('population by town-city.csv')
df['Size'] = ""
df.head
 
for index, row in df.iterrows():
  df.iloc[index, 0] = df.iloc[index, 0].replace("buadisp2011:", "")
  df.iloc[index, 0] = df.iloc[index, 0].replace(" BUASD", "")
  df.iloc[index, 0] = df.iloc[index, 0].replace(" BUA", "")
  string = df.iloc[index, 0]
  i = 0
  i = string.find(" - ")
  if i != -1:
    df.iloc[index, 0] = string[i+3:]
 
df.head

<bound method NDFrame.head of                  Area   mnemonic    2011    2012  ...    2017    2018    2019  Size
0               Kexby  E34000001     342     342  ...     364     352     337      
1     Bradfield Heath  E34000002     724     715  ...     821     838     831      
2             Huntley  E34000003    1108    1121  ...    1127    1108    1131      
3          Swinefleet  E34000004     787     758  ...     787     794     770      
4            Foulsham  E34000005    1021     995  ...    1017     987     965      
...               ...        ...     ...     ...  ...     ...     ...     ...   ...
6616       Pontardawe  W38000146   12349   12357  ...   12801   12970   13096      
6617          Newport  W38000147  128115  128855  ...  133353  134920  136078      
6618       Bryn Pydew  W38000149     448     477  ...     454     448     449      
6619      Pentre Berw  W38000150     392     397  ...     404     421     416      
6620      Mynydd-bach  W38000153     391     3

In [ ]:
# create a list of our conditions
conditions = [(df['2019'] >= 7500) & (df['2019'] <= 24999),
    (df['2019'] >= 25000) & (df['2019'] <= 59999)
    ]

# create a list of the values we want to assign for each condition
values = ['Small Town', 'Medium Town']

# create a new column and use np.select to assign values to it using our lists as arguments
df['Size'] = np.select(conditions, values)


size_df = df[df['Size'] != "0"]
size_df.reset_index(inplace=True, drop=True)

In [ ]:
size_df['Latitude'] = ""
size_df['Longitude'] = ""
 
 
for index, row in size_df.iterrows():
  latlong = None
  while latlong == None:
    g = geocoder.arcgis('{}, United Kingdom'.format(row['Area']))
    latlong = g.latlng
  lat = str(latlong[0])
  long = str(latlong[1])
  size_df.iloc[index, 13] = long[:10]
  size_df.iloc[index, 12] = lat[:9]

size_df.to_csv('towns_data.csv')
size_df.head()

In [ ]:
CLIENT_ID = '31FBQ4JXECZV0OKLQFEDFTWIKMA0CCKDTNJ3JFCMTQLNQHT3' # your Foursquare ID
CLIENT_SECRET = 'UHA3UUT4XUGCX5ABTQSH41WXLGNQUISCE35RFSFCG1HHP5LG' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 5000
categoryId= '52f2ab2ebcbc57f1066b8b46'
 
# create URL
def create_url(lat, long):
  url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{},&radius={}&limit={}&categoryId={}'.format(
      CLIENT_ID, 
      CLIENT_SECRET, 
      VERSION, 
      lat,
      long,
      radius, 
      LIMIT,
      categoryId)
  return url
 
 
res_df = pd.DataFrame(columns=['Area', 'Venue Category', 'Venue Name', 'Longitude', 'Latitude'])
temp_df = res_df
temp_df.at[1, :] = ""
for index, row in size_df.iterrows():
  temp = temp_df
  lat = row['Latitude']
  long = row['Longitude']
  url = create_url(lat, long)
  try:
    results = requests.get(url).json()["response"]['groups'][0]['items']
    time.sleep(1)
    for res in results:
      temp['Area'] = row['Area']
      temp['Venue Name'] = res['venue']['name']
      temp['Venue Category'] = res['venue']['categories'][0]['name']
      temp['Longitude'] = res['venue']['location']['labeledLatLngs'][0]['lng']
      temp['Latitude'] = res['venue']['location']['labeledLatLngs'][0]['lat']
      res_df = res_df.append(temp, ignore_index=True)
  except:
    print("Skipping {}".format(index))

In [ ]:
res_df = res_df[res_df['Venue Category'] == 'Supermarket']
res_df.to_csv('results.csv')

In [ ]:
print(res_df.shape)
res_df.drop_duplicates(inplace=True)

(4780, 5)


In [ ]:
res_df = pd.read_csv('results.csv')
print("Initial amount of unique names: {}".format(len(res_df['Venue Name'].unique().tolist())))

clean_df = res_df
search_name = ['orriso','o-op', 'press', 'aitrose', 'ermark', 'Tesco Superstore', 'Warehouse', 'Nisa Extra', 'Operative', 'Dpps']
new_name = ['Morrisons', 'Co-op', 'Tesco Express', 'Waitrose & Partners', 'Generic Supermarket', 'Tesco Extra', 'Iceland', 'Generic Supermarket', 'Co-op', 'Generic Supermarket']

for index, row in clean_df.iterrows():
  for num, name in enumerate(search_name):
    if name in row['Venue Name']:
      clean_df.loc[index, 'Venue Name'] = new_name[num]

print("Post Cleaning amount of unique names: {}".format(len(res_df['Venue Name'].unique().tolist())))

a = clean_df['Venue Name'].value_counts()
drop_list = []
for num, count in enumerate(a):
  if count < 10:
    drop_list.append(a.index[num])

#create drop series and specify types to keep
drop_index = []

#remove unwanted data
print("Removing unwanted Data")
for index, row in clean_df.iterrows():
  if row['Venue Name'] in drop_list:
    drop_index.append(index)

#clear unwanted data from 05 system type            
clean_df.drop(index = drop_index, axis =1, inplace=True)
clean_df.reset_index(inplace=True)
clean_df = clean_df.iloc[:, 2:]


print("Post Cleaning low count names: {}".format(len(res_df['Venue Name'].unique().tolist())))

print(clean_df['Venue Name'].unique().tolist())


['Waitrose & Partners', 'Lidl', 'Tesco', "Sainsbury's", 'Aldi', 'Morrisons', 'Booths', 'Tesco Extra', 'Asda', "Sainsbury's Local", 'Iceland', 'Co-op', 'Tesco Express', 'Generic Supermarket', 'Nisa Local']


In [ ]:
clean_df.to_csv('clean.csv')

,index,Unnamed: 0,Area,Venue Category,Venue Name,Longitude,Latitude
0,0,0,Dorchester (West Dorset),Supermarket,Waitrose & Partners,-2.435335,50.714134
1,1,1,Dorchester (West Dorset),Supermarket,Waitrose & Partners,-2.435335,50.714134
2,2,2,Dorchester (West Dorset),Supermarket,Lidl,-2.442251,50.718832
3,3,3,Dorchester (West Dorset),Supermarket,Waitrose & Partners,-2.466680,50.715337
4,4,4,Dorchester (West Dorset),Supermarket,Tesco,-2.443629,50.701867
...,...,...,...,...,...,...,...
4758,4811,4978,Porth,Supermarket,Lidl,-3.432096,51.644955
4759,4812,4979,Porth,Supermarket,Lidl,-3.345356,51.601489
4760,4813,4980,Porth,Supermarket,Asda,-3.454349,51.631156
4761,4814,4981,Pontardawe,Supermarket,Lidl,-3.851384,51.718200


In [ ]:
small_list = ["Sainsbury's Local", 'Co-op', 
              'Tesco Express', 'Nisa Local']

towns_df = pd.read_csv('towns_data.csv')
results_df = pd.read_csv('results.csv')
towns_df['Convenience Store'] = 0
towns_df['Supermarket'] = 0
for i, r in towns_df.iterrows():
  mask_df = results_df[results_df['Area'] == r['Area']]
  for index, row in mask_df.iterrows():
    if row['Venue Name'] in small_list:
      towns_df.loc[i, 'Convenience Store'] = towns_df.loc[i, 'Convenience Store'] + 1
    else:
      towns_df.loc[i, 'Supermarket'] = towns_df.loc[i, 'Supermarket'] + 1

towns_df.sort_values(by='Supermarket', inplace=True, ascending=False)
pd.set_option('display.max_rows', None)
towns_df

,Unnamed: 0,Area,mnemonic,2011,2012,2013,2014,2015,2016,2017,2018,2019,Size,Latitude,Longitude,Convenience Store,Supermarket
497,497,City of London,E35000758,7189,6405,5854,5952,6492,7046,7454,8480,9483,Small Town,51.520500,-0.097430,6,49
184,184,Cheadle,E34003425,11438,11388,11335,11313,11280,11303,11223,11232,11353,Small Town,53.395260,-2.212430,2,42
442,442,Cheadle,E35000569,13482,13570,13509,13645,13735,13697,13848,13913,14077,Small Town,53.395260,-2.212430,2,42
492,492,Droylsden,E35000734,24172,24180,24203,24001,24111,24278,24180,24235,24519,Small Town,53.479690,-2.145530,0,31
363,363,Denton,E35000254,36604,36576,36488,36457,36472,36249,36100,36203,36089,Medium Town,53.456010,-2.113680,0,25
564,564,Failsworth,E35001001,18842,18874,18840,18874,18712,18666,18726,18836,18843,Small Town,53.508460,-2.163370,0,24
436,436,Bootle,E35000538,51502,51283,51156,51070,51365,51757,51964,52079,52835,Medium Town,53.455950,-2.985950,0,23
712,712,Wallsend,E35001401,43985,44138,44327,44631,44484,44738,44791,45006,45441,Medium Town,54.991230,-1.533220,1,23
455,455,Darlaston,E35000597,18845,18937,19136,19469,19704,20012,20319,20678,20995,Small Town,52.566970,-2.034450,1,22
544,544,Cheadle Hulme,E35000932,24365,24410,24515,24712,24914,25207,25383,25391,25394,Medium Town,53.378060,-2.191030,2,22


In [63]:
import folium
import json
clean_df = pd.read_csv('clean.csv')
uk_map = folium.Map(location=(52.3555, 1.1743), zoom_start=6)

for index, row in clean_df.iterrows():
  lat = row['Latitude']
  long = row['Longitude']
  folium.Marker(location=[lat, long]).add_to(uk_map)

folium.GeoJson('wards.json', name='wards').add_to(uk_map)

folium.TopoJson(data='topo_wards.json', object_path='objects.wards',name='topo_wards').add_to(uk_map)

folium.LayerControl().add_to(uk_map)

uk_map.save('map.html')
uk_map


AttributeError: ignored

# New section

In [3]:
import pandas as pd

pd.read_json('topo_wards.json')

ValueError: ignored